# Solomaxxing: Data Olympics

Este notebook tiene el código para el proyecto de solomaxxing.

---

Colores corporativos:
- #13599B  # Blue

-  #2DCCCD  # Aqua

- #EE3A6A  # Pink

- #F35E61  # Coral

Imports

In [ ]:
# Official imports
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.feature_selection import mutual_info_regression, RFECV, RFE
from sklearn.model_selection import GridSearchCV, TimeSeriesSplit
from xgboost import XGBRegressor

# Inhouse imports
from utils import plot_dual_y

# Misc
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

Read data

In [ ]:
# Read
df = pd.read_csv(os.path.join('..', 'data', 'data_full.csv'))

# List of columns
cols_og_feats = list(df.columns[1:-1])

Set column types

In [ ]:
# x-ticks for plots
df['fecha_str'] = df['fecha'].str[:7]

# Cast fecha to date type
df['fecha'] = pd.to_datetime(df['fecha'], format='%Y-%m-%d')

## Feature engineering
1. Time features

In [ ]:
# Base features
df = df.assign(
    t = np.arange(len(df)),
    month = df['fecha'].dt.month,
    quarter = df['fecha'].dt.quarter,
    year = df['fecha'].dt.year
)

# From base features
df = df.assign(
    # Long term
    t2 = df['t'].pow(2),
    t3 = df['t'].pow(3),
    logt = np.log(df['t'] + 1),
    # Seasonality
    sin_month_1k = np.sin(2 * np.pi * df['month'] / 12),
    sin_month_2k = np.sin(2 * np.pi * 2 * df['month'] / 12),
    sin_month_3k = np.sin(2 * np.pi * 3 * df['month'] / 12),
    cos_month_1k = np.cos(2 * np.pi * df['month'] / 12),
    cos_month_2k = np.cos(2 * np.pi * 2 * df['month'] / 12),
    cos_month_3k = np.cos(2 * np.pi * 3 * df['month'] / 12),
    # Crises
    is_post_covid = df['fecha'].ge('2020-04-01').astype(int),
    is_post_gfc = df['fecha'].ge('2008-09-01').astype(int)
)

# Dummies
cols_season = ['month', 'quarter']
df = pd.concat(
    objs=[
        df,
        pd.get_dummies(df[cols_season], columns=cols_season, dtype=int)
    ],
    axis=1
)

2. Lags

In [ ]:
# Lag all columns
lags = [1, 2, 3, 6, 12]
_temp = df[cols_og_feats].shift(periods=lags)
_temp.columns = [
    '_'.join(col.split('_')[:-1]) + '_lag' + col.split('_')[-1]
    for col in _temp.columns
]  # Add _lagX suffix

# Fill nans with most recent value
for col in _temp.columns:
    col_og = col.split('_lag')[0]
    _temp[col] = _temp[col].fillna(df[col_og])
df = pd.concat([df, _temp], axis=1)
del _temp

3. Rolling functions

In [ ]:
windows = [3, 6, 9, 12]

# Silly functions
funs = ['mean', 'min', 'max', 'std']
for window in windows:
    _temp = (
        df[cols_og_feats]
        .rolling(window, min_periods=1, center=False)
        .agg(funs)
    )
    _temp.columns = [f"{'_'.join(col)}_w{window}" for col in _temp.columns]

    # Declare min/max ratio
    for col in cols_og_feats:
        _temp[f'{col}_minmax_w{window}'] = _temp[f'{col}_min_w{window}'].div(
            _temp[f'{col}_max_w{window}'].replace(0, 0.1)
        )
        
    # Drop min or max (only minmax)
    _temp = _temp.drop(
        columns=[col for col in _temp.columns if ('_min_w' in col) or ('_max_w' in col)]
    )

    # Append to df
    df = pd.concat([df, _temp], axis=1).fillna(0)

# EMAs
for span in windows[:2]:
    for col in cols_og_feats:
        df[f'{col}_ema{span}'] = df[col].ewm(span=span, adjust=False).mean()

## Bivariate Selection

Split data into features and target

In [ ]:
# Train test split
feats = cols_og_feats + [
    col for col in df.columns if ('_lag' in col) or ('_w' in col) or ('_ema' in col)
    or ('_sin' in  col) or ('_cos' in col) or ('is_post_' in col)
]
mask_train = df['fecha'].lt('2022-05-01')
X_train, y_train = df.loc[mask_train, feats], df.loc[mask_train, 'corporativa_mn']
X_test, y_test = df.loc[~mask_train, feats], df.loc[~mask_train, 'corporativa_mn']

# Mutial info scores
mi = mutual_info_regression(X_train, y_train, discrete_features='auto')
mi = dict(zip(feats, mi))

# Correlations
corr = X_train.corr()

# Correlation cutoff
CORR_CUTOFF = 0.8

# For every pair of features with |corr| > CORR_CUTOFF, drop the one with the lowest mutial info coef.
biv_drop = set()
for i, feat_i in enumerate(feats):
    for j, feat_j in enumerate(feats[i + 1:], start=i + 1):
        corr_ij = corr.iloc[i, j]
        if np.abs(corr_ij) > CORR_CUTOFF:
            # Drop feature with lower MI score
            if mi[feat_i] < mi[feat_j]:
                biv_drop.add(feat_i)
            else:
                biv_drop.add(feat_j)

# Drop biv_cols from all X dataset
X_train.drop(columns=biv_drop, inplace=True)
X_test.drop(columns=biv_drop, inplace=True)

# Print
print(f'Dropping {len(biv_drop)} columns using correlations + bivariate comparisons.')
print(f'Left with {len(feats) - len(biv_drop)} columns.')

## Hyperparameter Search

In [ ]:
# Params to try
grid = {
    'n_estimators': [150, 200, 250, 300],
    'max_depth': [1, 2, 3],
}

# CV split
tscv = TimeSeriesSplit(n_splits=8)

# Init search objects
xgb = XGBRegressor(
    objective='reg:squarederror',
    learning_rate=0.01,
    subsample=0.8,
    colsample_bytree=0.8,
    random_state=42
)
gcv = GridSearchCV(
    estimator=xgb,
    param_grid=grid,
    cv=tscv,
    scoring='neg_mean_squared_error',
    n_jobs=-1,
    verbose=0,
    return_train_score=True
)

# Fit search
gcv.fit(X_train, y_train)

# View results
res_gcv = pd.DataFrame(gcv.cv_results_)
res_gcv = (
    res_gcv[
        # Parameter cols
        [col for col in res_gcv.columns if 'param_' in col]
        # Eval cols
        + [
            'mean_train_score', 'std_train_score', 'mean_test_score', 'std_test_score',
            'rank_test_score'
        ]
    ]
    .assign(_score = res_gcv[['mean_test_score', 'std_train_score']].mean(axis=1))
    .sort_values('_score', ascending=False)
)
res_gcv.head(10)

## Feature Elimination

In [ ]:
# Init model
xgb_tuned = XGBRegressor(
    # learning_rate=0.01,
    # max_depth=2,
    # n_estimators=200,
    # subsample=0.8,
    # colsample_bytree=0.8,
    # random_state=42
    **gcv.best_estimator_.get_params()
)

# Init and fit RFECV
rfecv = RFECV(
    estimator=xgb_tuned,
    step=1,
    min_features_to_select=20,
    cv=tscv,
    scoring='neg_mean_squared_error',
    n_jobs=-1,
    verbose=0
)
rfecv.fit(X_train, y_train)

# View results
pd.DataFrame({
    'n_features': rfecv.cv_results_['n_features'],
    'mean_test_score': rfecv.cv_results_['mean_test_score'],
    'std_test_score': rfecv.cv_results_['std_test_score']
}).sort_values(by='mean_test_score', ascending=True).head(10)

In [ ]:
# Init and fit RFE
rfe = RFE(
    estimator=xgb_tuned,
    n_features_to_select=21,
    step=1
)
rfe.fit(X_train, y_train)

# Get results
selected_features = X_train.columns[rfe.support_]
print(f"Selected features ({len(selected_features)}):")
print(selected_features.tolist())

## Train tuned and pruned regressor

In [ ]:
# Final features
feats_final = list(X_train.columns[rfe.support_])

# Init and fit model
xgb_final = XGBRegressor(**xgb_tuned.get_params())
xgb_final.fit(X_train[feats_final], y_train)

# Make predictions
y_pred_all = xgb_final.predict(df[feats_final])
y_pred_test = xgb_final.predict(X_test[feats_final])

Overlaid plot

In [ ]:
plt.plot(
    df.loc[mask_train, 'fecha'],
    df.loc[mask_train, ['corporativa_mn']],
    lw=3,
    label='Observed'
)
plt.plot(df['fecha'], y_pred_all, color='#F35E61', label='Predicted')

# Aesthetics
plt.xlabel('Date')
plt.ylabel('Saldo (MN)')
plt.title('Predicción del saldo de la cartera corporativa')
tick_locs = np.arange(0, len(df), 12)
tick_labels = df['fecha_str'].iloc[tick_locs]
plt.xticks(ticks=df['fecha'].iloc[tick_locs], labels=tick_labels, rotation=45)
plt.legend()

# Show
plt.show()


Train-test plot

In [ ]:
plt.plot(df.loc[mask_train, 'fecha'], y_train, label='Observed')
plt.plot(df.loc[~mask_train, 'fecha'], y_pred_test, color='#F35E61', label='Forecast')

# Aesthetics
plt.legend()
plt.xlabel('Date')
plt.ylabel('Saldo (MN)')
plt.title('Predicción del saldo de la cartera corporativa')
tick_locs = np.arange(0, len(df), 12)
tick_labels = df['fecha_str'].iloc[tick_locs]
plt.xticks(ticks=df['fecha'].iloc[tick_locs], labels=tick_labels, rotation=45)

# Show
plt.show()

Final features VS target

In [ ]:
importances = dict(zip(xgb_final.feature_names_in_, xgb_final.feature_importances_))
feats_sorted = sorted(feats_final, key=lambda f: importances.get(f, 0), reverse=True)

# Sort according to importance!
for col in feats_sorted[:10]:
    plot_dual_y(df[mask_train], 'corporativa_mn', col, 'fecha_str')

Write preds

In [ ]:
df['pred'] = y_pred_all
pred = df.loc[~mask_train, ['fecha', 'pred']].reset_index(drop=True)
pred.to_csv(os.path.join('..', 'data', 'pred.csv'))